## Example: Properties of a Three-State Hidden Markov Models
This example will familiarize students with building and using [Hidden Markov Models](https://en.wikipedia.org/wiki/Hidden_Markov_model) to model random phenomena. In this example, we'll use the previous three-state model and assume the hidden states are mapped to the mood set: 

$$
\mathcal{S}\equiv\left\{\text{happy},\text{neutral},\text{sad}\right\}
$$

We'll then map the hidden states $s_{i}\in\mathcal{S}$ to outward manifestations of mood which are observable, as represented by [Emoji](https://en.wikipedia.org/wiki/Emoji).

## Setup
Let's load some packages that are required for the example by calling the `include(...)` function on our initialization file `Include.jl`:

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-13/L13a/Manifest.toml`


```julia
iterate(P::Array{Float64,2}; maxcount::Int = 100, ϵ::Float64 = 0.1) -> Array{Float64,2}
```
> Iteratively computes a stationary distribution. Computation stops if ||P_new - P|| < ϵ or the max number of iterations is hit. 

In [2]:
function iterate(P::Array{Float64,2}; maxcount::Int = 100, ϵ::Float64 = 0.1)::Array{Float64,2}

    # initialize -
    should_stop_loop = false
    counter = 1;
    P_new = P;
    
    # loop until tolerance is met, or we run out of iterations
    while (should_stop_loop == false)
        P_new = P^(counter+1);
        if (norm(P_new - P) < ϵ || counter > maxcount)
            should_stop_loop = true
        else
            counter += 1;
        end
    end

    # return -
    return P_new
end

iterate (generic function with 1 method)

#### Constants 
In the simulations below, we'll need some constant values that we set here. In particular, we set a value for the `number_of_hidden_states` variable, the `number_of_simulation_steps` variable (the number of steps that we take in a Markov Chain), and the `number_of_observable_states` variable:

In [3]:
number_of_hidden_states = 3;
number_of_observable_states = 3;
number_of_simulation_steps = 50000;

## Setup the Transition matrix $\mathbf{P}$
Let's start by specifying the transition matrix $\mathbf{P}$ for the _hidden states_ in the three-state Markov model:

<div>
    <center>
        <img src="figs/Fig-ThreeState-HMM-Schematic.svg" width="580"/>
    </center>
</div>

In this example we have `three` states $\mathcal{S}\equiv\left\{\text{happy},\text{neutral},\text{sad}\right\}
$ and the probability of moving between state $i\rightarrow{j}$, denoted as $p_{ij}$, is an element of the matrix $\mathbf{P} \in \mathbb{R}^{3\times{3}}$.

In [4]:
P = [
    0.05 0.95 0.0 ; # moves for state 1 = happy
    0.6 0.2 0.2 ; # moves for state 2 = neutral
    0.0 0.3 0.7 ; # moves for state 3 = sad
];

### Check: do the rows of the transition matrix $\mathbf{P}$ sum to `1`?
We know that the rows of the transition matrix $\mathbf{P}$ must sum to `1`, i.e., if we are in state $s_{i}\in\mathcal{S}$ at time $t$, then at time $t+1$ we have to be in $s_{j}\in\mathcal{S}$. 
* Let's check if the transition matrix $\mathbf{P}$ meets this criteria using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) by iterating over the rows of the transition matrix $\mathbf{P}$ and checking the sum of each row. If any row does not meet this criterion, an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) will be thrown.

In [5]:
for i ∈ 1:number_of_hidden_states
    @assert sum(P[i,:]) == 1
end

Now that we are sure that the transition matrix $\mathbf{P}$ is proper, we populate the `hidden_state_probability_dictionary`, which holds the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) modeling the transition probability for each hidden state $s\in\mathcal{S}$, i.e., the probability that we transition from state $i\rightarrow{j}$ in the next time step:

In [6]:
hidden_state_probability_dictionary = Dict{Int,Categorical}();
for i ∈ 1:number_of_hidden_states
    hidden_state_probability_dictionary[i] = Categorical(P[i,:])
end
hidden_state_probability_dictionary

Dict{Int64, Categorical{P} where P<:Real} with 3 entries:
  2 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.6, 0.2…
  3 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.0, 0.3…
  1 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.05, 0.…

## Compute the stationary distribution $\pi$
Let's compute the stationary distribution $\pi$ using the `iterate(...)` method defined above. This version of the method uses a `while-loop`; we iterate until the difference $||\mathbf{P}^{i+1} - \mathbf{P}^{i}|| < \epsilon$, or we run out of iterations (the iteration counter exceeds the `maxcount` argument).

In [7]:
π̄ = iterate(P, ϵ = 1e-9, maxcount = 10000) # iterative version of iterate

3×3 Matrix{Float64}:
 0.274809  0.435115  0.290076
 0.274809  0.435115  0.290076
 0.274809  0.435115  0.290076

Finally, create a [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) using the stationary probability of our Markov chain using the [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) package, save this distribution in the variable `d`:

In [8]:
d = Categorical(π̄[1,:]); # steady-state stationary distribution

## Setup the Emission Probability Matrix $\mathbf{E}$
Now that we have the stationary distribution for the hidden layer of our [Hidden Markov Model](https://en.wikipedia.org/wiki/Hidden_Markov_model) let's set up the emission probability matrix and save it in the `EPM` variable:

In [9]:
E = [
    0.90 0.05 0.05 ; # 1 happy (but sometimes we see other faces)
    0.05 0.90 0.05 ; # 2 neutral (but sometimes we see other faces)
    0.05 0.05 0.90 ; # 3 sad (but sometimes we see other faces)
];
# E = diagm(ones(3)) # we never have a missed guess ...

Populate the `emission_probability_dict`, which holds the [categorical distribution](https://en.wikipedia.org/wiki/Categorical_distribution) modeling the emission probability for each hidden state $s\in\mathcal{S}$, i.e., the probability of what output $o_{i}\in\mathcal{O}$ we expect to see if we are in $s\in\mathcal{S}$:

In [10]:
emission_probability_dict = Dict{Int,Categorical}()
for i ∈ 1:number_of_hidden_states
    emission_probability_dict[i] = Categorical(E[i,:])
end

In [11]:
emission_probability_dict

Dict{Int64, Categorical{P} where P<:Real} with 3 entries:
  2 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.05, 0.…
  3 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.05, 0.…
  1 => Categorical{Float64, Vector{Float64}}(support=Base.OneTo(3), p=[0.9, 0.0…

### 3. Simulate the output from the HMM
Let's implement the pseudo-code from the lecture, where each observable state corresponds to an [Emoji](https://en.wikipedia.org/wiki/Emoji). We store this relationship in the `observable_emoji_map` variable, which is a dictionary with keys corresponding to observable states $o\in\mathcal{O}$ and [Emoji](https://en.wikipedia.org/wiki/Emoji) values.

In [12]:
observable_emoji_map = Dict{Int,Any}();
observable_emoji_map[1] = `😄`;
observable_emoji_map[2] = `😐`;
observable_emoji_map[3] = `😞`;

__Simulation algorithm__: For `number_of_simulation_steps`, starting from some initial state $s\in\mathcal{S}$: 
* First we get the hidden state distribution from the `hidden_state_probability_dictionary`, we then generate a new state $s^{\prime}$, access the emission distribution from the `emission_probability_dict` that corresponds to $s^{\prime}$, and we generate a random observable output $o_{i}$.
* Next, we save both the hidden state $s^{\prime}$ and the output $o_{i}$ for this iteration in the `hidden_simulation_dict` and `output_simulation_dict` variables, respectively.
* Finally, we update the current state $s_{i}\leftarrow{s}^{\prime}$ and move onto the next iteration.

In [13]:
output_simulation_dict = Dict{Int,Any}()
hidden_simulation_dict = Dict{Int,Int}();
sᵢ = 1;
for i ∈ 1:number_of_simulation_steps

    # get the categorical distribution for sᵢ 
    dᵢ = hidden_state_probability_dictionary[sᵢ];
    
    # compute the *next* hidden state -
    s′ = rand(dᵢ);

    # next, compute what output we see from this state -
    oᵢ = emission_probability_dict[s′] |> o -> rand(o);

    # capture -
    hidden_simulation_dict[i] = s′
    output_simulation_dict[i] = observable_emoji_map[oᵢ]

    # update -
    sᵢ = s′;
end

In [14]:
foreach(i->println("$(hidden_simulation_dict[i]),$(output_simulation_dict[i])"), 1:20); # wow, that's nice ...

2,`😐`
1,`😄`
2,`😞`
1,`😄`
2,`😐`
1,`😄`
2,`😐`
3,`😐`
2,`😐`
3,`😞`
3,`😞`
3,`😞`
2,`😞`
1,`😄`
2,`😞`
3,`😞`
3,`😞`
3,`😞`
3,`😞`
3,`😞`


Let's do a quick test: what is the probability that we observe a particular value? We'll compute this by iterating over the simulation output and counting the times a `test_value` is encountered. We'll then estimate the probability as the number of positive samples divided by the total number of samples.

In [15]:
test_value = `😐`;
N₊ = 0;
for (key,value) ∈ output_simulation_dict
    if (value == test_value)
        N₊ += 1
    end
end
probability = N₊/number_of_simulation_steps;
println("We observe $(test_value) with probability = $(probability)")

We observe `😐` with probability = 0.42208


#### 